Problem postawiaonym w tym eksperymencie jest określenie jakości wina w skali od 0 do 10. Dane wzięto ze strony "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", a więcej informacji o nich jest pod linkiem http://archive.ics.uci.edu/ml/datasets/Wine+Quality. Pomimo tego, że jest 11 możliwych ocen jakości wina co mogło by sugerować klasyfikcaję 10 klasową, to z uwagi na to, że jakość wina jest wartością samą w sobie (nierozróżnienie win o różnicy jakości równej 1 to mały błąd, ale pomylenie win o jakości równej 8 to ograomny błąd), więc w tym zadaniu należy skorzystać z regresji.

Fragmet kodu pobierające dane do docelowego typu danych, na których odbędzie się uczenie - TabularDataset. Jest to azurowy typ danch, do którego można załadować tabelaryczne dane z pliku CSV, TSV, Parquet lub  za pomocą zapytań SQL. NAstępnie dane zapisano do typu danych DataFrame z biblioteki pandas by pokazać strukturę i wygląd danych.

In [20]:
import pandas as pd
from azureml.core import Dataset 

url="http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

aml_dataset = Dataset.Tabular.from_delimited_files(url, separator = ';')
full_df = aml_dataset.to_pandas_dataframe()
full_df



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.00,0.27,0.36,20.70,0.04,45.00,170.00,1.00,3.00,0.45,8.80,6
1,6.30,0.30,0.34,1.60,0.05,14.00,132.00,0.99,3.30,0.49,9.50,6
2,8.10,0.28,0.40,6.90,0.05,30.00,97.00,1.00,3.26,0.44,10.10,6
3,7.20,0.23,0.32,8.50,0.06,47.00,186.00,1.00,3.19,0.40,9.90,6
4,7.20,0.23,0.32,8.50,0.06,47.00,186.00,1.00,3.19,0.40,9.90,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.20,0.21,0.29,1.60,0.04,24.00,92.00,0.99,3.27,0.50,11.20,6
4894,6.60,0.32,0.36,8.00,0.05,57.00,168.00,0.99,3.15,0.46,9.60,5
4895,6.50,0.24,0.19,1.20,0.04,30.00,111.00,0.99,2.99,0.46,9.40,6
4896,5.50,0.29,0.30,1.10,0.02,20.00,110.00,0.99,3.34,0.38,12.80,7


Podział zbioru na dane testowe i dane treningowe.

In [21]:
train_dataset, test_dataset = aml_dataset.random_split(0.8, seed=1)

Pobranie do zmiennej Azurowego środowiska pracy do ML utworzonego w portalu Azure'owym, oraz zapisanie do zmiennej jednostki obliczającej, która będzie przetwarzała nasz model.

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core import Workspace

ws = Workspace.get('lab3WS', subscription_id='c01c3ddc-cf07-4160-8192-2a2476ee0151', resource_group='lab3')
ComputeTarget.list(ws)
amlcompute_cluster_name = "lab3CC"


Operacja sprawdzająca czy istnieje jednostka obliczniowa o wcześniej podanje nazwie i zapisanie jej do zmiennej

In [4]:
found = False
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     aml_remote_compute = cts[amlcompute_cluster_name]
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_DS11_V2",
                                                                 max_nodes = 20)
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


sprawdzenie istniejących możliwych funcji błędu dla zadania regresji.

In [5]:
from azureml.train.automl.utilities import get_primary_metrics
get_primary_metrics('regression')

['normalized_mean_absolute_error',
 'normalized_root_mean_squared_error',
 'spearman_correlation',
 'r2_score']

ustawienie parametrów procesu uczenia:
* compute_target - jednostka obliczeniowa
* task - typ zadania - klasyfikacja, regresja albo przewidywanie danych w szeregów czasowych
* primary_metric - wybór funkcji starty, tej która mówi jaka jest różnica pomiędzy obecnie wyznaczoną przez model wartością a wartością docelową
* experiment_timeout_minutes - czas zakończenia eksperymentu, gdy uczy się za długo                         
* training_data - dane trenigowe,
* label_column_name - kolumna danych, która jest warotścią, którą model ma się nauczyć przewiywać
* n_cross_validations - walidacja krzyżowa, jest to podział zbioru treningowego na n równych części, gdzie trenuje się n modeli, w których jedną z n części to zbiór validacyjny a cała reszta to zbiór treningowy. Na koniec uśrednia się wyniki tych n modeli i w ten sposób dane są bardziej odporne na podział danych zbiór treningowy i walidacyjny.                                                 
* enable_early_stopping - możliwośc zatrzymnia procesu ucznenia przed końcem odpowiedniej liczby iteracji, gdy wynik sie nie poprawia
* featurization - jest to automatyczna kalibracja danych, tzn. usuwanie tych rekordów gdzie jakiejś wartości nie ma, zamiana teksu na dane, które mogą podlegać uczeniu itp.


In [32]:
from azureml.train.automl import AutoMLConfig


automl_config = AutoMLConfig(compute_target = aml_remote_compute,
                             task = 'regression',
                             primary_metric = 'normalized_root_mean_squared_error',
                             experiment_timeout_minutes = 15,                            
                             training_data = train_dataset,
                             label_column_name = "quality",
                             n_cross_validations = 5,                                                  
                             enable_early_stopping=True,
                             featurization='auto',
                             )

stworznie nazwy dla eksperymentu, przeprowadzenie uczenia w danym środowisku dla danej konfiguracji ustawionej w poprzednum punkcie. Dodatkowo mierzy się czas uczenia i na koniec go wyświetla. Na dole wyświetlaja się sukcesywnie różne modele i ich wyniki. Im mniejszy wynik tym lepiej, bo nasz model mniej się myli

In [33]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "quality-wine-regression-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))


quality-wine-regression-01-26-2021-20
Running on remote.
No run_configuration provided, running on lab3CC with default configuration
Running on remote compute: lab3CC
Parent Run ID: AutoML_cc37dd10-6e13-4fe0-ade7-4ae1c56b261d

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  

Sprawdzenie stanu czy udało się zakończyć poprawnie uczenie.

In [34]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

parametry najlepszego modelu

In [37]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: quality-wine-regression-01-26-2021-20,
Id: AutoML_cc37dd10-6e13-4fe0-ade7-4ae1c56b261d_16,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                        

Zamiana danych testowych na DataFrame z biblioteki pandas i wydzielnie danych wejściowych, na których będą wyznaczane wyniki modelu i wyjściowych z którymi będize porównywany wynik modelu.

In [38]:
test_dataset_df = test_dataset.to_pandas_dataframe()
if 'quality' in test_dataset_df.columns:
    y_test_df = test_dataset_df['quality']

x_test_df = test_dataset_df.drop('quality', axis = 1)

Sprawdzenie wyników dla danych tesowych

In [39]:
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])


10 predictions: 
[5.44334316 6.51919566 5.98912183 5.78267409 5.03962486 6.08042763
 5.97142849 5.2151807  5.25955008 5.49487021]


Wyznacznie średniego błędu przewidywanych wartości dla zbioru testowego. 

In [43]:
from sklearn.metrics import mean_squared_error

print('RMSE:')
print(mean_squared_error(y_test_df, y_predictions))

acc = 0
counter = 0
for i, column in enumerate(y_predictions):
    if round(column) == y_test_df[i]:
        acc += 1
    counter += 1

print('Acc:')
print(acc/counter)

RMSE:
0.4389804726156803
Acc:
0.597948717948718


Średni wynik błędu to ok. 0,44 biorąc pod uwagę, a ułamek porawnie wyznaczonych ocen po zaokrągleniu przewidywań to 0,6. Z uwagi, że skala jest 0 do 10 a podziałka to 1, to znaczy, że zdecydwana większość wyników z 40 błędnie wyznaczonych ocen nie myliła się o więcej niż 1.

Wnioski z laboratorium:
* bardzo wygodne narzędzie pod kątem tego, że często architekt ML musi wykazać się dużą intuicją na temat architektury i algorytmów uczących a tu bardzo szybko prorównywane jest kilka algorytmów
* jednak bardzo dużym minusem jest bardzo słabo rozwinięty typ danych TabularDataset, w zasadzie poza funckją drop nie ma żadnych operacji na danych co byłoby bardzo przydatne, a konwersja z DF z biblioteki pandas jest na etapie eksperymentalnym i w zasadzie nie działa 